### CAA Mn content evolution over time

In [ ]:
import pickle
import numpy as np
import netCDF4 as nc
import os
from itertools import compress
import datetime as dt
import xarray as xr
import glob

%matplotlib inline

#### Parameters:

In [ ]:
# Domain dimensions:
imin, imax = 1479, 2179
jmin, jmax = 159, 799
isize = imax - imin
jsize = jmax - jmin

# Results folders:
folder_ref   = '/data/brogalla/run_storage/Mn-reference-202110/'
folder_clean = '/data/brogalla/run_storage/Mn-clean-ice-202110/'
folder_glac  = '/data/brogalla/run_storage/river-glacial-202112/'
folder_cont  = '/data/brogalla/run_storage/river-continental-202112/'

#### Load files:

In [ ]:
mesh       = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
lon        = np.array(mesh.variables['nav_lon'])
lat        = np.array(mesh.variables['nav_lat'])
mesh_bathy = np.array(mesh.variables['hdept'][0])
tmask      = np.array(mesh.variables['tmask'])[0,:,imin:imax,jmin:jmax]
land_mask  = np.ma.masked_where((tmask[:,:,:] > 0.1), tmask[:,:,:]) 
e1t_base   = np.array(mesh.variables['e1t'])[0,imin:imax,jmin:jmax]
e2t_base   = np.array(mesh.variables['e2t'])[0,imin:imax,jmin:jmax]
e3t        = np.array(mesh.variables['e3t_0'])[0,:,imin:imax,jmin:jmax]
e3t_masked = np.ma.masked_where((tmask[:,:,:] < 0.1), e3t)
nav_lev    = np.array(mesh.variables['nav_lev'])

e1t        = np.tile(e1t_base, (50,1,1))
e2t        = np.tile(e2t_base, (50,1,1))
volume     = e1t*e2t*e3t
area_base  = e1t_base*e2t_base
volume_masked = np.ma.masked_where((tmask[:,:,:] < 0.1), volume)
area_masked   = np.ma.masked_where((tmask[0,:,:] < 0.1), area_base)

In [ ]:
ref   = nc.Dataset(f'{folder_ref}ANHA12_ref-2002_20211012/ANHA12_EXH006_2002_monthly.nc',  'r')
lons  = np.array(ref.variables['nav_lon'])
lats  = np.array(ref.variables['nav_lat'])
depth = np.array(ref.variables['deptht'])

#### Functions:

In [ ]:
def calculate_diff(year, glac=False, cont=False, nosed=False):
    
    if glac:
        files_base  = np.sort(glob.glob(f'{folder_ref}ANHA12_ref-{year}_20211012/ANHA12_EXH006_5d*'))
        files_glac  = np.sort(glob.glob(f'{folder_glac}ANHA12_glacial-{year}_20211130/ANHA12_EXH006_5d*'))
        
        glacier_contribution   = np.empty((len(files_base),50,isize,jsize))
        for file_ind in range(0,len(files_base)):
            with xr.open_dataset(files_base[file_ind]) as base_fday:
                dmn_base  = base_fday['dissolmn'].values[0,:,:,:]
            with xr.open_dataset(files_glac[file_ind]) as glac_fday:
                dmn_glac  = glac_fday['dissolmn'].values[0,:,:,:]
                
            glacier_contribution[file_ind,:,:,:]   = dmn_glac - dmn_base # units: mol / L 
            
        return glacier_contribution
   
    elif cont:
        files_base  = np.sort(glob.glob(f'{folder_ref}ANHA12_ref-{year}_20211012/ANHA12_EXH006_5d*'))
        files_cont  = np.sort(glob.glob(f'{folder_cont}ANHA12_continental-{year}_20211130/ANHA12_EXH006_5d*'))
        
        continent_contribution = np.empty((len(files_base),50,isize,jsize))
        
        for file_ind in range(0,len(files_base)):
            with xr.open_dataset(files_base[file_ind]) as base_fday:
                dmn_base  = base_fday['dissolmn'].values[0,:,:,:]
            with xr.open_dataset(files_cont[file_ind]) as cont_fday:
                dmn_cont  = cont_fday['dissolmn'].values[0,:,:,:]
            
            continent_contribution[file_ind,:,:,:] = dmn_cont - dmn_base
            
        return continent_contribution
                
    elif nosed:
        files_base  = np.sort(glob.glob(f'{folder_ref}ANHA12_ref-{year}_20211012/ANHA12_EXH006_5d*'))
        files_nosed = np.sort(glob.glob(f'{folder_clean}ANHA12_clean-ice-{year}_20211023/ANHA12_EXH006_5d*'))
        
        sediment_contribution  = np.empty((len(files_base),50,isize,jsize))
    
    
        for file_ind in range(0,len(files_base)):
            with xr.open_dataset(files_base[file_ind]) as base_fday:
                dmn_base  = base_fday['dissolmn'].values[0,:,:,:]
            with xr.open_dataset(files_nosed[file_ind]) as nosed_fday:
                dmn_nosed  = nosed_fday['dissolmn'].values[0,:,:,:]

            sediment_contribution[file_ind,:,:,:]  = dmn_base - dmn_nosed
            
        return sediment_contribution
    else:
        print('choose an experiment from glac, cont, nosed')
        return

In [ ]:
def file_dates(year):
    #start_date and end_date are datetime objects
    start_date = dt.datetime(year,1,1)
    end_date   = dt.datetime(year,12,31,12)
    
    file_list1E = np.sort(os.listdir(f'{folder_ref}ANHA12_ref-{year}_20211012/'))
    file_list2 = np.sort(os.listdir('/data/brogalla/ANHA12/'))
    
    Vlist = [i[26:31]=='gridV' for i in file_list2]
    gridV_list = list(compress(file_list2, Vlist))
    dateV_list = [dt.datetime.strptime(i[14:25], "y%Ym%md%d") for i in gridV_list]
    gridV_file_list = list(compress(gridV_list, [V > start_date and V < end_date for V in dateV_list]))
    
    dates = [dt.datetime.strptime(i[14:25], "y%Ym%md%d") for i in gridV_file_list]
    
    return dates

#### Calculations:

Create mask for CAA

In [7]:
mask_ini_CAA  = np.ones(lon.shape) # full domain size
mask_ini_CAA[0:imin+10, :] = 0 # remove the boundaries
mask_ini_CAA[imax-10:, :]  = 0
mask_ini_CAA[:, 0:jmin]    = 0
mask_ini_CAA[:, jmax-20:]  = 0

# Define CAA using the longitudes
mask_CAA = np.ma.masked_where((mesh_bathy >= 800), mask_ini_CAA)
mask_CAA = np.ma.masked_where(lon < -130, mask_CAA)

mask_CAA_lons = np.ma.masked_where(mask_CAA==0, lon)
mask_CAA_lats = np.ma.masked_where(mask_CAA==0, lat)

mask_CAA_yr   = np.tile(mask_CAA[imin:imax,jmin:jmax], (73,50,1,1))

And mask for Canada Basin:

In [8]:
mask_ini_CB  = np.ones(lon.shape) # full domain size
mask_ini_CB[0:imin+10, :] = 0 # remove the boundaries
mask_ini_CB[imax-10:, :]  = 0
mask_ini_CB[:, 0:jmin]    = 0
mask_ini_CB[:, jmax-20:]  = 0

# Define Canada Basin using the 800 m contour and longitudes
mask_CB = np.ma.masked_where((mesh_bathy < 3200), mask_ini_CB)
mask_CB = np.ma.masked_where((lon > -80), mask_CB)

mask_CB_lons = np.ma.masked_where(mask_CB==0, lon)
mask_CB_lats = np.ma.masked_where(mask_CB==0, lat)

mask_CB_yr   = np.tile(mask_CB[imin:imax,jmin:jmax], (73,50,1,1))

In [9]:
mask_ini_fullCB  = np.ones(lon.shape) # full domain size
mask_ini_fullCB[0:imin+10, :] = 0 # remove the boundaries
mask_ini_fullCB[imax-10:, :]  = 0
mask_ini_fullCB[:, 0+10:jmin] = 0
mask_ini_fullCB[:, jmax-20:]  = 0

# Define fullCB using the longitudes
mask_fullCB = np.ma.masked_where((mesh_bathy < 800), mask_ini_fullCB)
mask_fullCB = np.ma.masked_where((lon > -60), mask_fullCB)
mask_fullCB = np.ma.masked_where((lon > -100) & (lat < 78), mask_fullCB)

mask_fullCB_lons = np.ma.masked_where(mask_fullCB==0, lon)
mask_fullCB_lats = np.ma.masked_where(mask_fullCB==0, lat)

mask_fullCB_yr   = np.tile(mask_fullCB[imin:imax,jmin:jmax], (73,50,1,1))

Calculate the glacial time series:

In [10]:
timeseries_glac_CAA = []
timeseries_glac_CBf = []
timeseries_glac_CBc = []

for year in range(2002, 2020):
    print(year)
    glacier = calculate_diff(year, glac=True)
    glacier_CAA = np.ma.masked_where(mask_CAA_yr==0   , glacier)
    glacier_CBf = np.ma.masked_where(mask_fullCB_yr==0, glacier)
    glacier_CBc = np.ma.masked_where(mask_CB_yr==0    , glacier)
    
    # multiply by gridbox volume before summing...
    # (mol / L) * (m3) * (1000 L / m3) --> moles in the given region
    for fday in range(0,73):        
        timeseries_glac_CAA.append(np.ma.sum(glacier_CAA[fday,:,:,:]*volume_masked*1e3))
        timeseries_glac_CBf.append(np.ma.sum(glacier_CBf[fday,:,:,:]*volume_masked*1e3))
        timeseries_glac_CBc.append(np.ma.sum(glacier_CBc[fday,:,:,:]*volume_masked*1e3))
        
pickle.dump((timeseries_glac_CAA, timeseries_glac_CBf, timeseries_glac_CBc), \
            open('/ocean/brogalla/GEOTRACES/pickles/FWC_glacial_timeseries.pickle','wb'))

2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


Calculate the continental time series:

In [11]:
timeseries_cont_CAA = []
timeseries_cont_CBf = []
timeseries_cont_CBc = []

for year in range(2002, 2020):
    print(year)
    continental = calculate_diff(year, cont=True)
    continental_CAA = np.ma.masked_where(mask_CAA_yr==0   , continental)
    continental_CBf = np.ma.masked_where(mask_fullCB_yr==0, continental)
    continental_CBc = np.ma.masked_where(mask_CB_yr==0    , continental)
    
    # multiply by gridbox volume before summing...
    # (mol / L) * (m3) * (1000 L / m3) --> moles in the given region
    for fday in range(0,73):        
        timeseries_cont_CAA.append(np.ma.sum(continental_CAA[fday,:,:,:]*volume_masked*1e3))
        timeseries_cont_CBf.append(np.ma.sum(continental_CBf[fday,:,:,:]*volume_masked*1e3))
        timeseries_cont_CBc.append(np.ma.sum(continental_CBc[fday,:,:,:]*volume_masked*1e3))
        
pickle.dump((timeseries_cont_CAA, timeseries_cont_CBf, timeseries_cont_CBc), \
            open('/ocean/brogalla/GEOTRACES/pickles/FWC_continental_timeseries.pickle','wb'))

2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


Sea ice:

In [10]:
timeseries_nosed_CAA = []
timeseries_nosed_CBf = []
timeseries_nosed_CBc = []

for year in range(2002, 2020):
    print(year)
    cleanice = calculate_diff(year, nosed=True)
    cleanice_CAA = np.ma.masked_where(mask_CAA_yr==0   , cleanice)
    cleanice_CBf = np.ma.masked_where(mask_fullCB_yr==0, cleanice)
    cleanice_CBc = np.ma.masked_where(mask_CB_yr==0    , cleanice)
    
    # multiply by gridbox volume before summing...
    # (mol / L) * (m3) * (1000 L / m3) --> moles in the given region
    for fday in range(0,73):        
        timeseries_nosed_CAA.append(np.ma.sum(cleanice_CAA[fday,:,:,:]*volume_masked*1e3))
        timeseries_nosed_CBf.append(np.ma.sum(cleanice_CBf[fday,:,:,:]*volume_masked*1e3))
        timeseries_nosed_CBc.append(np.ma.sum(cleanice_CBc[fday,:,:,:]*volume_masked*1e3))
        
pickle.dump((timeseries_nosed_CAA, timeseries_nosed_CBf, timeseries_nosed_CBc), \
            open('/ocean/brogalla/GEOTRACES/pickles/FWC_cleanice_timeseries.pickle','wb'))

2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
